Pipelines for model trainng @author Tim Copeland

In [9]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

In [10]:
ommit  = ('id','type','bond_id','trade_price')
target = '20151'
# Create a class to select numerical columns
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, ommit):
        self.ommit = ommit
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        #add year_qnum column
        X['year_qnum'] = X['date'].apply(lambda x: int(str(x[0:4]) + str(X['qnum'])))
        
        ret_X = X[X['year_qnum'] < target]
        attribute_names = [i for i in list(X.columns) if i not in ommit]
        return ret_X[attribute_names].values

SGDClassifier with both hinge loss (SVM) and log loss (logistic regression)

In [11]:
#pre processes data for regression
full_pipeline = Pipeline([        
        ('selector', DataFrameSelector(ommit)),
        ('imputer', Imputer(strategy="median")),
        ('std_scaler', StandardScaler())
    ])

def svd_create_full_pipeline_with_predictor(predictor):
    pipe = Pipeline([
        ("preparation", full_pipeline),
        ("linear", predictor)
    ])
    return pipe

def svd_run_describe_predictors(predictors, X, y):
    res = pd.DataFrame()
    for name, predictor in predictors.items():
        pipe = svd_create_full_pipeline_with_predictor(predictor)
        scores = cross_val_score(pipe, X, y,scoring='neg_mean_squared_error', cv=10) #10-fold cross-validation
        print(pd.Series(scores).describe())
        print('')
        res[str(name)] = pd.Series(scores).describe()
    return res

In [ ]:
predictors = {'SGD_log':  SGDClassifier(loss = 'log')}
res = run_describe_predictors(predictors, X, y)
print(res)

In [ ]:
predictors = {'SGD_hinge':SGDClassifier()}
res = run_describe_predictors(predictors, X, y)
print(res)